<!--NAVIGATION-->
< [Merge Daily JSON into DataFrame](For_The_Bit_01-03_Merge_Daily_JSON_into_DataFrame.ipynb) | [Contents](For_The_Bit_00-Introduction.ipynb) | [Add Exercise Flags to Intraday](For_The_Bit_01-05_Add_Exercise_Flags_to_Intraday.ipynb) >

# For The Bit

## 1 - Getting my `fitbit` data 

### Part IV.  Programmatically fetch all exercise logs

The exercise logs contain information about runs, bike rides, hikes, workouts, yoga sessions-- or any other type of activity currently supported through `fitbit`.  Most activities (for me at least) are triggered by the user-- that is I toggled my Charge 2 to the activity menu and then select an activity.  However, some activities are automatically classified, like walks and runs.  These automatically classified activities do no have associated GPS information, but otherwise behave the same as far as the API is concerned.

In [1]:
import json
import pandas as pd
import requests
import os.path
import time

From my API access code (this is a secret!)

In [2]:
with open('../curl_key.txt', 'r') as f:
    secret_key = f.readline()

In [3]:
headers = {'Authorization': 'Bearer {}'.format(secret_key)}

Mimic a directory structure for each user, with subdirectories as below:

In [4]:
! tree -d ../data/gully/

../data/gully/
├── exercise
│   ├── HR
│   ├── logs
│   └── tcx
├── intraday
│   ├── HR
│   ├── sleep
│   └── steps
└── weight
    ├── logs
    └── ts

11 directories


I think there's only one list of all activities ever performed.  The API call is:

`https://api.fitbit.com/1/user/-/activities/list.json`

Employ a file naming convention that preserves metadata:  
- **exercise\_log\_{running\_number}\_{logid}.json**  

Where `running_number` is a made up index padded with 5 leading zeros starting at 00000 for my first ever activity.  
The `logid` is from the API query itself, and is also the number used to request the `TCX`.

### Make a big *while* loop, download

The structure of this API call is a little different than it was for the intraday data.  The default is to return a *paginated* list of exercise activities.  I think the best strategy for now is to avoid this paginated structure, and simply have a unique json file for each exercise activity log.  This will make it easier to match exercise activities and `gpx` data later on, for example.  So the strategy here should be to perform our first query "blind", then all subsequent queries should use the `pagination` keyword to find the next exercise activity.

In [5]:
#! rm ../data/gully/exercise/activity_list.json

I got my Charge 2 on: `2016-12-17`

In [6]:
params =  {'user-id':'-',
           'afterDate':'2016-12-16T00:00:00',
           'sort':'asc',
           'limit':'1',
           'offset':'0'}

Zeroth query.

### Get the most recent exercise date

After you already have one file, you can make downloads relative to the most recent log, to avoid repeat downloading.

In [7]:
lsdir = os.listdir('../data/gully/exercise/logs/')

In [8]:
most_recent_log = lsdir[-1]

In [9]:
most_recent_log

'exercise_log_7439891935.json'

In [10]:
with open('../data/gully/exercise/logs/{}'.format(most_recent_log)) as f:
    mr_json = json.load(f)

st = mr_json['activities'][0]['startTime']
afterDate = st[:-10]
params['afterDate'] = afterDate

Get the *next* most recent:

In [11]:
query_str = 'https://api.fitbit.com/1/user/-/activities/list.json'
# Only make a query if we don't already have the file.
response = requests.get(query_str, params=params, headers=headers)
logId = response.json()['activities'][0]['logId']
local_file = 'exercise_log_{}.json'.format(logId)
full_path = '../data/gully/exercise/logs/'+local_file
print(local_file, response.status_code)
with open(full_path, 'w') as f:
    json.dump(response.json(), f)

exercise_log_7439891935.json 200


In [12]:
while response.json()['pagination']['next'] != '':
    query_str = response.json()['pagination']['next']
    response = requests.get(query_str, headers=headers)
    logId = response.json()['activities'][0]['logId']
    local_file = 'exercise_log_{}.json'.format(logId)
    full_path = '../data/gully/exercise/logs/'+local_file
    print(local_file, response.status_code)
    with open(full_path, 'w') as f:
        json.dump(response.json(), f)

The end.